# SIR-Hawkes
This repository hosts the scripts and datasets for SIR-Hawkes project.

# Citation
The project was introduced in this [paper](https://arxiv.org/abs/1711.01679):
```
Marian-Andrei Rizoiu, Swapnil Mishra, Quyu Kong, Mark Carman, Lexing Xie. 2018. SIR-Hawkes: Linking Epidemic Models and Hawkes Processes to Model Diffusions in Finite Populations. . In WWW 2018: The 2018 Web Conference, April 23–27, 2018, Lyon, France. ACM, New York, NY, USA, 10 pages. https://doi.org/10.1145/3178876.3186108
```

# SIR-Hawkes tutorial

### required packages:
    - nloptr
    - parallel

### 1. We need to first load all required packages for simulation and modeling cascades.

In [1]:
library(parallel)
source('rscripts/functions-SIR-HawkesN.R')

### 2. We then simulate 20 stochastic SIR realizations

In [2]:
params.S <- c(N = 1300, I.0 = 300, gamma = 0.2, beta = 1)
nsim <- 20
simdat <- replicate(
    n = nsim,
    generate.stochastic.sir(params = params.S, Tmax = 11)
)

Current simulation time: 11.065 / 11.000 (S=26, I=213, R=1061, C=1274).
--> Simulation done!
Current simulation time: 11.009 / 11.000 (S=22, I=219, R=1059, C=1278).
--> Simulation done!
Current simulation time: 11.029 / 11.000 (S=15, I=220, R=1065, C=1285).
--> Simulation done!
Current simulation time: 11.023 / 11.000 (S=12, I=199, R=1089, C=1288).
--> Simulation done!
Current simulation time: 11.052 / 11.000 (S=18, I=237, R=1045, C=1282).
--> Simulation done!
Current simulation time: 11.015 / 11.000 (S=12, I=238, R=1050, C=1288).
--> Simulation done!
Current simulation time: 11.019 / 11.000 (S=11, I=228, R=1061, C=1289).
--> Simulation done!
Current simulation time: 11.003 / 11.000 (S=20, I=223, R=1057, C=1280).
--> Simulation done!
Current simulation time: 11.018 / 11.000 (S=17, I=248, R=1035, C=1283).
--> Simulation done!
Current simulation time: 11.007 / 11.000 (S=16, I=216, R=1068, C=1284).
--> Simulation done!
Current simulation time: 11.036 / 11.000 (S=19, I=212, R=1069, C=1281)

In [3]:
# let's take a look at the simulated data. One simulation is identified as four components: relative times,
# susceptible population size at each time,  infected population size at each time and recovered population
# size at each time
print(simdat[, 1], digits = 5)

$time
   [1] 0.0000e+00 3.5918e-04 2.2878e-03 3.8287e-03 3.9998e-03 4.1508e-03
   [7] 9.8626e-03 9.8790e-03 1.1473e-02 1.2105e-02 1.7112e-02 1.9561e-02
  [13] 1.9564e-02 3.1235e-02 3.2681e-02 3.4605e-02 3.5299e-02 3.5709e-02
  [19] 3.6031e-02 4.2125e-02 4.3850e-02 5.1969e-02 5.5464e-02 6.4834e-02
  [25] 7.0180e-02 7.3086e-02 8.2212e-02 8.4678e-02 8.5325e-02 8.8507e-02
  [31] 9.7862e-02 9.8442e-02 1.0749e-01 1.0894e-01 1.0965e-01 1.1864e-01
  [37] 1.1918e-01 1.2927e-01 1.3199e-01 1.3257e-01 1.4205e-01 1.4544e-01
  [43] 1.4649e-01 1.4677e-01 1.4879e-01 1.5532e-01 1.6008e-01 1.6266e-01
  [49] 1.6269e-01 1.7216e-01 1.7252e-01 1.7579e-01 1.8084e-01 1.8403e-01
  [55] 1.8438e-01 1.8597e-01 1.8892e-01 1.8897e-01 1.9006e-01 1.9406e-01
  [61] 2.0178e-01 2.0440e-01 2.0741e-01 2.1231e-01 2.1254e-01 2.1375e-01
  [67] 2.1599e-01 2.1882e-01 2.1929e-01 2.2229e-01 2.3648e-01 2.4265e-01
  [73] 2.4312e-01 2.4448e-01 2.4508e-01 2.4663e-01 2.4914e-01 2.4935e-01
  [79] 2.5396e-01 2.5913e-01 2.6208e-01 2.648

### 3. Fit stochastic SIR on simulated cascades 

In [4]:
# initial fitting point for each execution
params.fit.start <- c(N = 0.1, I.0 = 0.1, gamma = 0.1, beta = 0.1)

.cl <- makeCluster(spec = min(nsim, detectCores()), type = 'FORK')
results <- parSapply(cl = .cl, X = 1:nsim, FUN = function(i) {
    mysim <- as.data.frame(simdat[, i])
    return(fit.stochastic.sir(mysim, params.fit.start))
})
stopCluster(.cl)

# reconstruct result data format
res <- as.data.frame(results[1,])
names(res) <- 1:nsim             
res <- as.data.frame(t(res))     
res$ll <- unlist(results[2,])
complete_res <- res

In [5]:
# let's see how well parameters were retreived
prnt <- rbind(params.S[c('N', 'I.0', 'gamma', 'beta')], 
              apply(X = complete_res[, c('N', 'I.0', 'gamma', 'beta')], MARGIN = 2, FUN = median),
              apply(X = complete_res[, c('N', 'I.0', 'gamma', 'beta')], MARGIN = 2, FUN = sd))
rownames(prnt) <- c('theoretical', 'median', 'sd')
print(prnt[, c('N', 'I.0', 'gamma', 'beta')], digits = 2)

                 N I.0  gamma  beta
theoretical 1300.0 300 0.2000 1.000
median      1283.5 300 0.2009 0.995
sd             5.4   0 0.0054 0.031


### 4. Fit HawkesN on simulated cascades

In [6]:
## get the means at given time points, to be able to compare to deterministic
simhistory <- sapply(X = 1:nsim, FUN = function(i) {
  history.S <- SIR2HAWKES.stochastic.event.series(state = simdat[,i])  
})

In [22]:
## these are the theoretical parameters
params.H <- c(K = 5, c = 0.001, theta = 0.2, N = 1300)

# start point 
params.fit.start <- c(K = 1, c = 0.1, theta = 0.1, N = 1000)

## fit the event series with HawkesN
.cl <- makeCluster(spec = min(20, detectCores()), type = 'FORK')
results <- parSapply(cl = .cl, X = 1:nsim, FUN = function(i) {
  history.S <- as.data.frame(simhistory[,i])
  fitted.model <- fitSeries(history = history.S, params.fit.start)
})
stopCluster(.cl)
res <- as.data.frame(sapply(results, FUN = function(res) {return(res[['par']])}))
names(res) <- 1:nsim
res <- data.frame(t(res))


prnt <- rbind(params.H, 
              apply(X = res, MARGIN = 2, FUN = median, na.rm = T),
              apply(X = res, MARGIN = 2, FUN = sd, na.rm = T))
rownames(prnt) <- c('theoretical', 'median', 'sd')
print(prnt[, c('K', 'theta', 'c', 'N')], digits = 2)

res$gamma <- res$theta
res$beta <- res$K * res$theta
prnt <- rbind(params.S[c('N', 'gamma', 'beta')], 
              apply(X = res[, c('N', 'gamma', 'beta')], MARGIN = 2, FUN = mean, na.rm = T),
              apply(X = res[, c('N', 'gamma', 'beta')], MARGIN = 2, FUN = sd, na.rm = T))
rownames(prnt) <- c('theoretical', 'median', 'sd')
print(prnt[, c('N', 'gamma', 'beta')], digits = 2)

              K theta     c    N
theoretical 5.0 0.200 0.001 1300
median      5.4 0.182 0.100 1299
sd          1.2 0.048 0.000   11
               N gamma  beta
theoretical 1300 0.200 1.000
median      1300 0.193 0.990
sd            11 0.048 0.052


# License
Both dataset and code are distributed under the Creative Commons Attribution-NonCommercial 4.0 International (CC BY-NC 4.0) license, a copy of which can be obtained following this link. If you require a different license, please contact us at Marian-Andrei@rizoiu.eu or Lexing.Xie@anu.edu.au.
